In [2]:
import torch
import torch.nn as nn
import numpy as np
from tqdm.autonotebook import tqdm
from polar import PolarCode,get_frozen
from reliability_sequence import Reliability_Sequence
from utils import errors_ber,errors_bler


C:\Users\pratyush\AppData\Local\Temp\ipykernel_10308\70640425.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
n = 4
N = 2**n 
K = 9
snr = 1
batch_size = 1
num_samples = 1000


In [3]:

def create_data(num_samples, batch_size, n, K, snr):
    msg_bits_list = []
    bpsk_list = []
    codeword_list = []
    corrupted_codeword_list = []

    # Loop to generate data samples
    for i in range(num_samples):
        msg_bits = (torch.rand(batch_size, K) > 0.5).float()
        bpsk = 1 - 2 * msg_bits

        polar = PolarCode(n, K, Fr=None, use_cuda=True, hard_decision=True)
        codeword = polar.encode(bpsk)
        corrupted_codewords = polar.channel(codeword, snr)

        msg_bits_list.append(msg_bits.cpu().numpy())
        bpsk_list.append(bpsk.cpu().numpy())
        codeword_list.append(codeword.cpu().numpy())
        corrupted_codeword_list.append(corrupted_codewords.cpu().numpy())

    filename = f"polar_dataset_N{2**n}_K{K}_SNR{snr}_bs{batch_size}.npz"
    np.savez(f"polar_dataset_N{2**n}_K{K}_SNR{snr}_bs{batch_size}.npz", msg_bits=msg_bits_list, corrupted_codeword=corrupted_codeword_list,bpsk = bpsk_list,codeword=codeword_list)
    print(f"Dataset saved as {filename}")
    

In [4]:
create_data(num_samples,batch_size,n,K,snr)

Dataset saved as polar_dataset_N8_K3_SNR1_bs1.npz


In [4]:
df = np.load('data\polar_dataset_N16_K9_SNR1_bs1.npz')

In [6]:
from tqdm.autonotebook import tqdm
Fr = get_frozen(N, K,rs=Reliability_Sequence)
polar = PolarCode(n, K, Fr = Fr,use_cuda=False,hard_decision=True)
device = 'cpu'
ber_SC_total=0
bler_SC_total=0
x=10000
for bpsk_bits, corrupted_codeword in tqdm(zip(df['bpsk'][:x], df['corrupted_codeword'][:x]),total=len(df['bpsk'][:x])):
    bpsk_tensor = torch.tensor(bpsk_bits, dtype=torch.float32,device=device)
    corrupted_codeword_tensor = torch.tensor(corrupted_codeword, dtype=torch.float32,device=device)

    SC_llrs, decoded_SC_msg_bits = polar.sc_decode_new(corrupted_codeword_tensor, snr=snr)
    ber_SC = errors_ber(bpsk_tensor,decoded_SC_msg_bits.sign()).item()
    bler_SC = errors_bler(bpsk_tensor,decoded_SC_msg_bits.sign()).item()

    ber_SC_total+=ber_SC
    bler_SC_total+=bler_SC

100%|██████████| 1000/1000 [00:14<00:00, 68.29it/s]


In [7]:
decoded_SC_msg_bits, corrupted_codeword_tensor, bpsk_tensor

(tensor([[ 1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.]]),
 tensor([[ 2.4066,  0.3885,  1.5644,  1.8708,  2.0070,  0.9793,  1.5226,  2.1272,
           0.1533,  1.7197, -0.5839, -0.1718, -0.6325, -1.8706, -0.7579, -2.6515]]),
 tensor([[-1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.]]))

In [8]:
ber_SC_total/len(df['msg_bits'][:x]),bler_SC_total/len(df['msg_bits'][:x])

(0.2923333366438746, 0.815)

### LSTM LOGIC1

In [9]:
class LSTMDecoder(nn.Module):
    def __init__(self,N,K,hidden_size,rs):
        super().__init__()
        self.N = N
        self.K = K
        self.hidden_size = hidden_size

        self.frozen_positions = get_frozen(self.N, self.K, rs)
        self.frozen_mask = torch.full((N,), -1, dtype=torch.int8)
        # self.frozen_mask = torch.zeros(N,dtype=torch.bool)
        self.frozen_mask[self.frozen_positions] = 1
        self.lstms = nn.ModuleList([
            nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=1, batch_first=True)
            for _ in range(N)
        ])
        self.fcs= nn.ModuleList([
            nn.Linear(hidden_size, 1) for _ in range(N)
        ])
    
    def forward(self,corrupted_codeword):
        batch_size = corrupted_codeword.size(0)
        device = corrupted_codeword.device
        x = corrupted_codeword.unsqueeze(-1) if corrupted_codeword.dim() == 2 else corrupted_codeword

        h0 = [torch.zeros(1, batch_size, self.hidden_size, device=device) for _ in range(self.N)]
        c0 = [torch.zeros(1, batch_size, self.hidden_size, device=device) for _ in range(self.N)]

        decoded_outputs = []
        for i in range(self.N):
            output, (h0[i], c0[i]) = self.lstms[i](x, (h0[i], c0[i]))
            print(output.shape)
            decoded_bits = self.fcs[i](output).squeeze(-1)
            decoded_outputs.append(decoded_bits)
            x = corrupted_codeword + (self.frozen_mask.float().to(device)*decoded_bits.sign()) #current logic
            x = x.unsqueeze(-1)
        
        decoded_outputs = torch.stack(decoded_outputs,dim=1)
        # non_frozen_mask = ~self.frozen_mask
        non_frozen_mask = (self.frozen_mask == -1).nonzero(as_tuple=True)[0]

        final_predictions = decoded_outputs[:,-1,non_frozen_mask]

        return decoded_outputs, final_predictions


### LSTM LOGIC2

In [ ]:
class LSTMDecoder2(nn.Module):
    def __init__(self,N,K,hidden_size,rs):
        super().__init__()
        self.N = N
        self.K = K
        self.hidden_size = hidden_size

        self.frozen_positions = get_frozen(self.N, self.K, rs)
        self.frozen_mask = torch.full((N,), 0, dtype=torch.int8)
        self.frozen_mask[self.frozen_positions] = 1
        
        self.lstms = nn.ModuleList([
            nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=1, batch_first=True)
            for _ in range(N)
        ])
        self.fcs= nn.ModuleList([
            nn.Linear(hidden_size+1, 1) for _ in range(N) # 1 for the frozen bit
        ])
    
    def forward(self,corrupted_codeword):
        batch_size = corrupted_codeword.size(0)
        device = corrupted_codeword.device
        x = corrupted_codeword.unsqueeze(-1) if corrupted_codeword.dim() == 2 else corrupted_codeword

        h0 = [torch.zeros(1, batch_size, self.hidden_size, device=device) for _ in range(self.N)]
        c0 = [torch.zeros(1, batch_size, self.hidden_size, device=device) for _ in range(self.N)]

        decoded_outputs = []
        for i in range(self.N):
            x_t = x[:,i,:].unsqueeze(1)
            lstm_output, (h0[i], c0[i]) = self.lstms[i](x_t, (h0[i], c0[i]))
        
            frozen_bit = self.frozen_mask[i].float().to(device) #single predicted bit
            frozen_bit = frozen_bit.unsqueeze(0).expand(batch_size, 1, 1)  #[B,N, prediction_value]          
            lstm_output_with_frozen = torch.cat([lstm_output, frozen_bit], dim=-1) #[B,N,H+1]
            predicted_bit = self.fcs[i](lstm_output_with_frozen).squeeze(-1) #[B,1]
            decoded_outputs.append(predicted_bit)
            # x = corrupted_codeword + (self.frozen_mask.float().to(device)*decoded_bits.sign()) #current logic
            # x = torch.cat([x, predicted_bit.unsqueeze(-1)], dim=2)[:,:,-1:]
            x[:,i,:] = predicted_bit #x = [B,N,1]

        
        decoded_outputs = torch.stack(decoded_outputs,dim=1)
        # non_frozen_mask = ~self.frozen_mask
        non_frozen_mask = (self.frozen_mask == 0).nonzero(as_tuple=True)[0]

        final_predictions = decoded_outputs[:,-1,non_frozen_mask]

        return decoded_outputs, final_predictions



## Improved LSTM

In [9]:
class ImprovedLSTMDecoder(nn.Module):
    def __init__(self, N, K, hidden_size, rs):
        super().__init__()
        self.N = N
        self.K = K
        self.hidden_size = hidden_size

        self.frozen_positions = get_frozen(self.N, self.K, rs)
        self.frozen_mask = torch.full((N,), -1, dtype=torch.int8)
        self.frozen_mask[self.frozen_positions] = 1

        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=2, bidirectional=True, batch_first=True)
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size * 2, num_heads=4)
        self.fc = nn.Linear(hidden_size * 2, 1)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.residual = nn.Linear(1, hidden_size * 2)

    def forward(self, corrupted_codeword):
        batch_size = corrupted_codeword.size(0)
        device = corrupted_codeword.device
        x = corrupted_codeword.unsqueeze(-1) if corrupted_codeword.dim() == 2 else corrupted_codeword

        h0 = torch.zeros(2 * 2, batch_size, self.hidden_size, device=device)
        c0 = torch.zeros(2 * 2, batch_size, self.hidden_size, device=device)

        lstm_out, _ = self.lstm(x, (h0, c0))
        lstm_out = self.layer_norm(lstm_out)

        # Attention mechanism
        attn_output, _ = self.attention(lstm_out, lstm_out, lstm_out)
        attn_output = self.layer_norm(attn_output + lstm_out)

        # Residual connection
        residual = self.residual(x)
        attn_output = attn_output + residual

        decoded_bits = self.fc(attn_output).squeeze(-1)

        # Handle frozen bits
        decoded_bits[:, self.frozen_positions] = 0

        non_frozen_mask = (self.frozen_mask == -1).nonzero(as_tuple=True)[0]
        return decoded_bits[:, non_frozen_mask]


In [10]:

device = 'cuda'
model = ImprovedLSTMDecoder(N=N, K=K, hidden_size=32, rs=Reliability_Sequence).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

In [11]:
def calculate_ber_bler(predictions, targets):
    """ Calculate BER and BLER using errors_ber and errors_bler functions. """
    ber = errors_ber(targets, predictions.sign()).item()
    bler = errors_bler(targets, predictions.sign()).item()
    return ber, bler

In [12]:
num_epochs = 30
batch_size = 32
patience = 5  # Early stopping patience
best_loss = float('inf')
trigger_times = 0

data_len = len(df['msg_bits'][:900])
for epoch in range(num_epochs):
    model.train()
    total_loss, total_ber, total_bler = 0, 0, 0

    for msg_bits, corrupted_codeword in tqdm(zip(df['msg_bits'][:900], df['corrupted_codeword'][:900]), total=data_len):
        msg_tensor = torch.tensor(msg_bits, dtype=torch.float32).to(device)
        corrupted_codeword_tensor = torch.tensor(corrupted_codeword, dtype=torch.float32).to(device)
        
        optimizer.zero_grad()
        final_predictions = model(corrupted_codeword_tensor)
        # print(final_predictions.shape)
        loss = criterion(final_predictions, msg_tensor)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        
        total_loss += loss.item()
        ber, bler = calculate_ber_bler((final_predictions >= 0).float(), msg_tensor)
        total_ber += ber
        total_bler += bler

    # Compute average loss and error rates
    avg_loss = total_loss / data_len
    avg_ber = total_ber / data_len
    avg_bler = total_bler / data_len

    # Early stopping check
    if avg_loss < best_loss:
        best_loss = avg_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

    # Logging progress every 3 epochs
    if (epoch + 1) % 3 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, BER: {avg_ber:.4f}, BLER: {avg_bler:.4f}")

print("Training complete.")


100%|██████████| 900/900 [00:05<00:00, 162.09it/s]


Epoch 3/30, Loss: 0.6446, BER: 0.4265, BLER: 0.9944


100%|██████████| 900/900 [00:05<00:00, 161.34it/s]


Epoch 6/30, Loss: 0.6089, BER: 0.3828, BLER: 0.9800


100%|██████████| 900/900 [00:05<00:00, 164.31it/s]


...

Epoch 24/30, Loss: 0.3659, BER: 0.1684, BLER: 0.7167


100%|██████████| 900/900 [00:05<00:00, 159.24it/s]


Epoch 27/30, Loss: 0.3264, BER: 0.1363, BLER: 0.6367


100%|██████████| 900/900 [00:05<00:00, 165.48it/s]

Epoch 30/30, Loss: 0.2942, BER: 0.1165, BLER: 0.5889
Training complete.


In [13]:
ber_total,bler_total = 0,0
test_loader = df['msg_bits'][900:]
count = 0
with torch.no_grad():
    for msg_bits, corrupted_codeword in zip(df['msg_bits'][900:], df['corrupted_codeword'][900:]):
        count += 1
        msg_tensor = torch.tensor(msg_bits, dtype=torch.float32).to(device)
        corrupted_codeword_tensor = torch.tensor(corrupted_codeword, dtype=torch.float32).to(device)

        final_predictions = model(corrupted_codeword_tensor)
        if (count%20==0):
            print(f'final_predictions {(final_predictions>=0).float()}')
            print(f'msg_tensor {msg_tensor}')
            if((final_predictions>0).float()==msg_tensor).all():
                print('decoded correctly')
            else:
                print('decoded incorrectly')
            print('----------')
        ber,bler = calculate_ber_bler((final_predictions>=0).float(),msg_tensor)
        ber_total += ber
        bler_total += bler
    avg_ber = ber_total / len(test_loader)
    avg_bler = bler_total / len(test_loader)
    print(f"Test Results - BER: {avg_ber:.4f}, BLER: {avg_bler:.4f}")

final_predictions tensor([[1., 1., 1., 0., 0., 0., 1., 1., 0.]], device='cuda:0')
msg_tensor tensor([[1., 0., 1., 0., 1., 0., 1., 1., 0.]], device='cuda:0')
decoded incorrectly
----------
final_predictions tensor([[0., 1., 0., 1., 0., 1., 1., 0., 0.]], device='cuda:0')
msg_tensor tensor([[0., 1., 1., 1., 1., 1., 1., 0., 0.]], device='cuda:0')
decoded incorrectly
----------
final_predictions tensor([[0., 0., 0., 1., 0., 1., 1., 0., 1.]], device='cuda:0')
msg_tensor tensor([[0., 0., 0., 1., 0., 0., 1., 0., 1.]], device='cuda:0')
decoded incorrectly
----------
final_predictions tensor([[1., 0., 0., 0., 1., 1., 0., 1., 1.]], device='cuda:0')
msg_tensor tensor([[1., 1., 0., 1., 1., 1., 1., 1., 1.]], device='cuda:0')
decoded incorrectly
----------
final_predictions tensor([[1., 1., 1., 1., 0., 0., 0., 0., 1.]], device='cuda:0')
msg_tensor tensor([[1., 1., 1., 0., 0., 0., 0., 0., 1.]], device='cuda:0')
decoded incorrectly
----------
Test Results - BER: 0.3711, BLER: 1.0000
